In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import numpy as np
import pandas as pd
import random

In [3]:
df=pd.read_csv("KMeansDataset.csv")
df.head()

,Feature1,Feature2
0,1.149014,1.958521
1,1.194307,2.456909
2,0.929754,1.929759
3,1.473764,2.230230
4,0.859158,2.162768


In [4]:
df1=df.copy()
l=len(df)

##### Calculating distance between two instances(rows)

In [5]:
def dist(x,y):    
    val=0
    l=len(x)
    for i in range(l):
        val+=(x[i] - y[i])**2

    return (val**0.5)

##### Calculating Sum of Squared distances between centroid and it's corresponding clustered points 

In [6]:
def WCSS(centroids,store,k):
    val=0
    for i in range(k):
        for index,key in enumerate(store):
            if key==i:
                val+=(dist(centroids[i],df.iloc[index,]))**2

    return val**0.5

### K Means

##### Finding initial centroids

In [7]:
def initial_centroids(k):
    
    indices=list(range(0, l-1))   ## Create a list of numbers from 0 to l-1
    random.shuffle(indices)       ## Shuffle the list
    indices=indices[0:k]          ## Get the first k numbers from the shuffled list

    points=[df.iloc[item,] for item in indices]     ## Selecting corresponding rows
    
    return points

##### Main K-Means Function

In [8]:
def cluster(k,tol,n_iter):
    
    centroids=initial_centroids(k)

    store=[]        ## This holds centroid number of each row sequentially inside it.
    for i in range(l):
        index=np.argmin([dist(centroids[j],df.iloc[i,]) for j in range(k)])  ## Choosing the centroid number from which a row is at minimum distance
        store.append(index)
        
    w_old=WCSS(centroids,store,k)   ## Calculating WCSS with old centroids

    count=0     ## variable to track total number of required iterations
    while True:
        count+=1
        
        centroids=[]     ## This is for calculating new centroids
        for i in range(k):      
            x=np.mean([df.iloc[index,0] for index,val in enumerate(store) if val==i])   
            y=np.mean([df.iloc[index,1] for index,val in enumerate(store) if val==i])
            centroids.append([x,y])

        store=[]   ## This holds new-centroid number of each row sequentially inside it.
        for i in range(l):
            index=np.argmin([dist(centroids[j],df.iloc[i,]) for j in range(k)])
            store.append(index)
        
        w_new=WCSS(centroids,store,k)   ## Calculating WCSS with new centroids
        
        if (abs(w_old-w_new)<tol and count>=n_iter):   ## Checking tolerance level and number of iterations
            break

        w_old=w_new

    return [centroids,store]

In [9]:
k=5
tol=0.01
n_iter=20

call=cluster(k,tol,n_iter)
centroids=call[0]
clusters=call[1]
print(centroids)
print(clusters)

[[np.float64(0.8742295523807113), np.float64(1.6700791254313752)], [np.float64(5.481222618832473), np.float64(8.01691142661409)], [np.float64(8.990862310941985), np.float64(1.7297123649568364)], [np.float64(8.945074831042108), np.float64(2.236370144544734)], [np.float64(1.0052873869848624), np.float64(2.139743623249379)]]
[np.int64(4), np.int64(4), np.int64(4), np.int64(4), np.int64(4), np.int64(0), np.int64(0), np.int64(0), np.int64(4), np.int64(0), np.int64(4), np.int64(0), np.int64(4), np.int64(4), np.int64(0), np.int64(4), np.int64(0), np.int64(0), np.int64(0), np.int64(4), np.int64(4), np.int64(4), np.int64(0), np.int64(4), np.int64(0), np.int64(4), np.int64(4), np.int64(4), np.int64(0), np.int64(4), np.int64(4), np.int64(0), np.int64(4), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1), np.int64(1),

### K Means ++

##### Finding initial centroids

In [10]:
def initial_centroids(k):
    points=[]
    
    r=random.randint(0,l-1)  ## Generating random number to choose the initial centroid 
    c1=df.iloc[r,]
    points.append(c1)
    
    dist_c1=[dist(c1,df.iloc[i,]) for i in range(l)]  ## Distances between rows and the initial centroid
    c2=df.iloc[np.argmax(dist_c1),]                   ## Choosing centroid 2 whose distance is maximum from the 1st centroid
    points.append(c2)

    for j in range(k-2):                                                            ## Here j runs upto k-2 since 2 initial centroids have been obtained
        temp=[min([dist(item,df.iloc[i,]) for item in points]) for i in range(l)]   ## Calculating distances from each centroid then finding minimum 
        new=df.iloc[np.argmax(temp),]                                               ## distance for all rows and among the minimum distances choosing 
        points.append(new)                                                          ## the row which gives maximum among them.

    return points

##### Main K-Means ++ Function

In [11]:
def cluster(k,tol,n_iter):
    
    centroids=initial_centroids(k)

    store=[]        ## This holds centroid number of each row sequentially inside it.
    for i in range(l):
        index=np.argmin([dist(centroids[j],df.iloc[i,]) for j in range(k)])  ## Choosing the centroid number from which a row is at minimum distance
        store.append(index)
        
    w_old=WCSS(centroids,store,k)   ## Calculating WCSS with old centroids

    count=0     ## variable to track total number of required iterations
    while True:
        count+=1
        
        centroids=[]     ## This is for calculating new centroids
        for i in range(k):      
            x=np.mean([df.iloc[index,0] for index,val in enumerate(store) if val==i])   
            y=np.mean([df.iloc[index,1] for index,val in enumerate(store) if val==i])
            centroids.append([x,y])

        store=[]   ## This holds new-centroid number of each row sequentially inside it.
        for i in range(l):
            index=np.argmin([dist(centroids[j],df.iloc[i,]) for j in range(k)])
            store.append(index)
        
        w_new=WCSS(centroids,store,k)   ## Calculating WCSS with new centroids
        
        if (abs(w_old-w_new)<tol and count>=n_iter):   ## Checking tolerance level and number of iterations
            break

        w_old=w_new

    return [centroids,store]

In [12]:
k=5
tol=0.01
n_iter=20

call=cluster(k,tol,n_iter)
centroids=call[0]
clusters=call[1]
print(centroids)
print(clusters)

[[np.float64(0.9536585430498937), np.float64(1.9547242756241046)], [np.float64(8.945074831042108), np.float64(2.236370144544734)], [np.float64(5.530308630725165), np.float64(8.272855075690137)], [np.float64(5.440317608921898), np.float64(7.803625052384051)], [np.float64(8.990862310941985), np.float64(1.7297123649568364)]]
[np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(0), np.int64(2), np.int64(3), np.int64(2), np.int64(2), np.int64(2), np.int64(3), np.int64(3), np.int64(2), np.int64(3), np.int64(3), np.int64(2), np.int64(2), np.int64(2), np.int64(3), np.int64(3), np.int64(2), np.int64(3), np.int64(3), np.int64(3),